<a href="https://colab.research.google.com/github/trnhatnam/Projets_IA/blob/main/People_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Détection de personnes - notebook écrit TRINH Nhat-Nam

workflow :
- extraction de la base de données sur kaggle
- conversion sous format coco
- entrainement sur RF-DETR

In [ ]:
!pip install kaggle


In [ ]:
from google.colab import files

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nhatnamtrinh","key":"36478a6e5d22b60af1cd52a25d4c01c5"}'}

In [1]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [2]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download adilshamim8/people-detection

Dataset URL: https://www.kaggle.com/datasets/adilshamim8/people-detection
License(s): Attribution-NoDerivatives 4.0 International (CC BY-ND 4.0)
 99% 1.93G/1.94G [00:11<00:00, 108MB/s] 
100% 1.94G/1.94G [00:11<00:00, 177MB/s]


In [ ]:
!ls

kaggle.json  people-detection.zip  sample_data


In [6]:
!mkdir dataset

mkdir: cannot create directory ‘dataset’: File exists


In [7]:
!unzip people-detection -d dataset

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: dataset/train/train/2010_005494_jpg.rf.1d921a649beb2274af3cdf6776aa358d.jpg  
  inflating: dataset/train/train/2010_005494_jpg.rf.695eafd03e6ea75a5ad6c8b8ca830938.jpg  
  inflating: dataset/train/train/2010_005494_jpg.rf.927c318646d7c0611b04697d69d0e3be.jpg  
  inflating: dataset/train/train/2010_005512_jpg.rf.13f5d37fdf9cc51c16793383addced52.jpg  
  inflating: dataset/train/train/2010_005512_jpg.rf.b6023800c60bed07e7c5704b21e6a463.jpg  
  inflating: dataset/train/train/2010_005512_jpg.rf.fedb4d425ea255a3257648cf5d0a3384.jpg  
  inflating: dataset/train/train/2010_005515_jpg.rf.7651507057a6f4de76e0ffaa0688036b.jpg  
  inflating: dataset/train/train/2010_005515_jpg.rf.9b71380f9c44a20659c2c7fcf9a2f464.jpg  
  inflating: dataset/train/train/2010_005515_jpg.rf.f2471546b5e7c031dd750b6bd0de3103.jpg  
  inflating: dataset/train/train/2010_005518_jpg.rf.793622cc24f0a0ae7cb132f46e7f5803.jpg  
  inflating:

In [8]:
!find ./dataset/train/ -type f -name '*.csv'

./dataset/train/train/_annotations.csv


# Pré-configuration

In [9]:
import os
from google.colab import userdata

os.environ["ROBOFLOW_API_KEY"] = userdata.get("ROBOFLOW_API_KEY")

In [10]:
!pip install -q rfdetr supervision roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 10.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.8/372.8 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Récréation du dataset en COCO dataset

In [11]:
from tqdm import tqdm
import json
import numpy as np
import pandas as pd

In [12]:
def create_coco_format(annotations):
  coco_format = dict()


  coco_format['info'] = {
        "year": "2020",
        "version": "1",
        "description": "Exported from roboflow.ai",
        "contributor": "Roboflow",
        "url": "https://app.roboflow.ai/datasets/hard-hat-sample/1",
        "date_created": "2000-01-01T00:00:00+00:00"
    }
  coco_format['categories'] = list()
  coco_format['images'] =  list()
  coco_format['annotations'] = list()

  # catégorie
  for i, cat in enumerate(annotations['class'].unique()):
    elem_cat = dict()
    elem_cat['id'] = i
    elem_cat['name'] = cat
    elem_cat['supercategory'] = 'none'

    coco_format['categories'].append(elem_cat)


  unique_id_image_train = annotations.groupby('filename').ngroup()
  added_index = np.zeros(len(unique_id_image_train))

  for index,row in tqdm(annotations.iterrows()):
    # images
    if added_index[unique_id_image_train[index]] == 0:
      elem_image = dict()
      elem_image['id'] = int(unique_id_image_train[index])
      elem_image['file_name'] = row['filename']
      elem_image['width'] = row['width']
      elem_image['height'] = row['height']

      added_index[unique_id_image_train[index]] = 1
      coco_format['images'].append(elem_image)

    # annotations
    elem_annotation = dict()
    elem_annotation['id'] = index
    elem_annotation['image_id'] = int(unique_id_image_train[index])
    elem_annotation['category_id'] = 0
    elem_annotation['bbox'] = [row['xmin'], row['ymin'], row['xmax']-row['xmin'], row['ymax']-row['ymin']]
    elem_annotation['area'] = (row['xmax']-row['xmin']) * (row['ymax']-row['ymin'])
    elem_annotation['iscrowd'] = 0

    coco_format['annotations'].append(elem_annotation)
  return coco_format

In [13]:
train_coco_format = create_coco_format(pd.read_csv('./dataset/train/train/_annotations.csv'))
test_coco_format = create_coco_format(pd.read_csv('./dataset/test/test/_annotations.csv'))
val_coco_format = create_coco_format(pd.read_csv('./dataset/valid/valid/_annotations.csv'))

100082it [00:17, 5827.90it/s]
2783it [00:00, 5332.76it/s]
10683it [00:01, 5794.85it/s]


## Création du dataset sous format coco

In [14]:
!mkdir dataset_coco
!mkdir dataset_coco/train
!mkdir dataset_coco/test
!mkdir dataset_coco/valid

In [15]:
!find dataset/train/train -type f -name "*.jpg" -exec cp {} dataset_coco/train \;
with open('dataset_coco/train/_annotations.coco.json', 'w') as f:
  json.dump(train_coco_format, f)

In [16]:
!find dataset/test/test -type f -name "*.jpg" -exec cp {} dataset_coco/test \;
with open('dataset_coco/test/_annotations.coco.json', 'w') as f:
  json.dump(test_coco_format, f)

In [17]:
!find dataset/valid/valid -type f -name "*.jpg" -exec cp {} dataset_coco/valid \;
with open('dataset_coco/valid/_annotations.coco.json', 'w') as f:
  json.dump(val_coco_format, f)

In [18]:
!mkdir out

In [ ]:
from rfdetr import RFDETRSmall

model = RFDETRSmall()

model.train(
    dataset_dir="dataset_coco",
    epochs=10,
    batch_size=4,
    grad_accum_steps=4,
    lr=1e-4,
    output_dir="out",
    #checkpoint = "checkpoint.pth"
)

rf-detr-small.pth: 100%|██████████| 368M/368M [00:17<00:00, 21.5MiB/s]


Using a different number of positional encodings than DINOv2, which means we're not loading DINOv2 backbone weights. This is not a problem if finetuning a pretrained RF-DETR model.
Using patch size 16 instead of 14, which means we're not loading DINOv2 backbone weights. This is not a problem if finetuning a pretrained RF-DETR model.
Loading pretrain weights
TensorBoard logging initialized. To monitor logs, use 'tensorboard --logdir out' and open http://localhost:6006/ in browser.
Not using distributed mode
git:
  sha: N/A, status: clean, branch: N/A

Namespace(num_classes=1, grad_accum_steps=4, amp=True, lr=0.0001, lr_encoder=0.00015, batch_size=4, weight_decay=0.0001, epochs=10, lr_drop=100, clip_max_norm=0.1, lr_vit_layer_decay=0.8, lr_component_decay=0.7, do_benchmark=False, dropout=0, drop_path=0.0, drop_mode='standard', drop_schedule='constant', cutoff_epoch=0, pretrained_encoder=None, pretrain_weights='rf-detr-small.pth', pretrain_exclude_keys=None, pretrain_keys_modify_to_load=N

Epoch: [0]  [  0/829]  eta: 2:03:12  lr: 0.000100  class_error: 0.00  loss: 8.4932 (8.4932)  loss_ce: 0.9500 (0.9500)  loss_bbox: 0.3920 (0.3920)  loss_giou: 0.6564 (0.6564)  loss_ce_0: 0.8918 (0.8918)  loss_bbox_0: 0.5198 (0.5198)  loss_giou_0: 0.7450 (0.7450)  loss_ce_1: 0.9374 (0.9374)  loss_bbox_1: 0.4494 (0.4494)  loss_giou_1: 0.6831 (0.6831)  loss_ce_enc: 0.9287 (0.9287)  loss_bbox_enc: 0.5397 (0.5397)  loss_giou_enc: 0.7999 (0.7999)  loss_ce_unscaled: 0.9500 (0.9500)  class_error_unscaled: 0.0000 (0.0000)  loss_bbox_unscaled: 0.0784 (0.0784)  loss_giou_unscaled: 0.3282 (0.3282)  cardinality_error_unscaled: 6.2500 (6.2500)  loss_ce_0_unscaled: 0.8918 (0.8918)  loss_bbox_0_unscaled: 0.1040 (0.1040)  loss_giou_0_unscaled: 0.3725 (0.3725)  cardinality_error_0_unscaled: 6.2500 (6.2500)  loss_ce_1_unscaled: 0.9374 (0.9374)  loss_bbox_1_unscaled: 0.0899 (0.0899)  loss_giou_1_unscaled: 0.3415 (0.3415)  cardinality_error_1_unscaled: 6.2500 (6.2500)  loss_ce_enc_unscaled: 0.9287 (0.9287) 

KeyboardInterrupt: 